In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
filename = "models/phi-3-16.gguf"
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(filename, device_map="tpu:0")

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.padding_side = "right"

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import jax
import jax.numpy as jnp
prompt = """<|user|>
What is 3940 * 3892?<|end|>
<|assistant|>
"""
tokens = tokenizer.encode(prompt)
def tokens_to_array(tokens):
    token_array = jnp.asarray(tokens)
    if len(token_array) >= llama.mesh.shape["dp"]:
        token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq")
    return token_array

In [5]:
from micrlhf.llama import LlamaBlock
from functools import partial


layer_source = 10
def make_get_resids(llama, layer_target):
    get_resids = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer_target
                                                                              ).apply(lambda x:
        pz.nn.Sequential([
            pz.de.TellIntermediate.from_config(tag=f"resid_pre"),
            x
        ])
    )
    get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
    return get_resids
jittify = lambda x: partial(jax.jit(lambda lr, *args, **kwargs: lr(*args, **kwargs)[1][0].value), x)
get_resids_initial = make_get_resids(llama, layer_source)
get_resids_initial = jittify(get_resids_initial)

In [6]:
layer_target = 20
taker = jittify(make_get_resids(llama, layer_target).select().at_instances_of(LlamaBlock).apply_with_selected_index(
    lambda i, x: x if i >= layer_source else pz.nn.Identity()
).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

In [9]:
import dataclasses
import jax.numpy as jnp
from tqdm.auto import trange
import optax


inputs = llama.inputs.from_basic_segments(tokens_to_array([tokens]))
resid_initial = get_resids_initial(inputs)
bs = 16
seed = 9
iterations = 1000
scale = 10
def get_loss(rep):
    rep = pz.nx.wrap(rep, "batch", "embedding")
    rep = pz.nx.nmap(lambda a: a / jnp.linalg.norm(a) * scale)(rep.untag("embedding")).tag("embedding")
    resid = pz.nx.nmap(lambda x, y: x.at[-1].add(y))(resid_initial[{"batch": 0}].untag("seq"), rep).tag("seq")
    resid_out = taker(dataclasses.replace(inputs, tokens=resid,
                                          attention_mask=inputs.attention_mask,
                                          positions=inputs.positions))[{"seq": -1}]
    diffs = pz.nx.nmap(lambda x: (x[:, None] - x[None, :]) ** 2)(resid_out.untag("batch")).tag("batch1", "batch2")
    diffs = diffs.untag("embedding").sum()  # ** 0.5
    loss = diffs.data_array.mean()
    return -loss, (resid_out,)
start_add = jax.random.normal(jax.random.key(seed),
                              (bs, resid_initial.named_shape["embedding"]), dtype=resid_initial.dtype)

optimizer = optax.chain(
    optax.clip_by_global_norm(1.0),
    optax.adam(1e-2),
    optax.zero_nans(),
    
)
lwg = jax.value_and_grad(get_loss, has_aux=True)
# lwg = jax.jit(lwg)

# @partial(jax.jit, donate_argnums=(0, 1))
def train_step(addition, opt_state):
    (loss, (resid_out,)), grad = lwg(addition)
    updates, opt_state = optimizer.update(grad, opt_state, addition)
    addition = optax.apply_updates(addition, updates)
    return loss, addition, opt_state, dict(resid_out=resid_out)


In [10]:
addition = start_add
opt_state = optimizer.init(addition)
for _ in (bar := trange(iterations)):
    loss, addition, opt_state, aux = train_step(addition, opt_state)
    bar.set_postfix(loss=loss)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [29]:
from micrlhf.utils.activation_manipulation import add_vector
from micrlhf.sampling import sample
import numpy as np

layer = layer_source
repeat = 4
act_add = add_vector(llama,
                     jnp.tile(addition / jnp.linalg.norm(addition, axis=-1, keepdims=True) * scale, (repeat, 1)),
                     layer)

texts, cached = sample(act_add, tokenizer, prompt,
       batch_size=bs * repeat, do_sample=True, return_model=True)
np.array(texts).reshape(repeat, bs).T.tolist()

  0%|          | 0/46 [00:00<?, ?it/s]

In [31]:
from micrlhf.utils.load_sae import get_sae
from micrlhf.utils.ito import grad_pursuit
resid_out = aux["resid_out"]
refusal = (resid_out[{"batch": -4}] - resid_out.untag("batch").mean()).unwrap("embedding")
sae = get_sae(20, 5)
k = 2
w, recon = grad_pursuit(refusal, sae["W_dec"], k, pos_only=False)
i = jax.lax.top_k(jnp.abs(w), k)[1]
((recon - refusal) ** 2).mean(), i, w[i]

(<jax.Array(22.22797, dtype=float32)>,
 <jax.Array([40212, 19938], dtype=int32)>,
 <jax.Array([-58.395702, -50.326153], dtype=float32)>)

In [58]:
from micrlhf.utils.load_sae import get_sae
from micrlhf.utils.ito import grad_pursuit
refusal_steer = addition[-4]
sae_steer = get_sae(10, 4)
k = 16
w, recon_steer = grad_pursuit(refusal_steer, sae_steer["W_dec"], k, pos_only=False)
i = jax.lax.top_k(jnp.abs(w), k)[1]
((recon_steer - refusal_steer) ** 2).mean(), i, w[i]

(<jax.Array(1.0269325, dtype=float32)>,
 # jax.Array int32(16,) [≥4_722, ≤43_323] nonzero:16
   Array([38392,  9105, 43323,  4722,  7395, 22994, 21968, 13932, 27525,
           4764,  9626,  8265, 35330, 32930, 18141, 11405], dtype=int32)
 ,
 # jax.Array float32(16,) ≈-1.4 ±3.8 [≥-4.5, ≤5.0] nonzero:16
   Array([ 5.0286074, -4.5097766, -4.400113 , -4.3175597,  4.2154174,
          -4.198103 ,  4.121102 ,  4.04647  , -4.0310316, -3.9342709,
          -3.8903906,  3.8370857, -3.7827194, -3.7405024, -3.6709757,
          -3.5558388], dtype=float32)
 )

In [56]:
# layer = layer_target
# repeat = 64
# vector = refusal
# # vector = recon
# act_add = add_vector(llama,
#                     (vector / jnp.linalg.norm(vector, axis=-1, keepdims=True))[None, :] * jnp.linspace(-200, 200, repeat)[:, None],
#                      layer)

layer = layer_source
repeat = 64
# vector = addition[-4]
vector = recon_steer
act_add = add_vector(llama,
                     (vector / jnp.linalg.norm(vector, axis=-1, keepdims=True))[None, :] * jnp.linspace(-scale * 2, scale * 2, repeat)[:, None],
                     layer)

sample(act_add, tokenizer, prompt,
       batch_size=repeat, do_sample=False)

  0%|          | 0/46 [00:00<?, ?it/s]

['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n---------\nFirst, multiply ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n---------\nFirst, multiply ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n-----\n2',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n2',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n2',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n2',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you multiply the two numbers together:\n\n3940 * 3892 = 15,368,8',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you simply multiply the two numbers together:\n\n3940 * 3892 = 15,368,',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you multiply the two numbers together:\n\n3940 * 3892 = 15,368,8',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n1',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you multiply the two numbers together:\n\n3940 * 3892 = 15,344,8',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n1',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you can use the standard algorithm for multiplication:\n\n```\n  3940\nx 3892\n------\n ',
 "<s><|user|> What is 3940 * 3